<a href="https://colab.research.google.com/github/hien25661/learn-machine-learning-in-two-months/blob/master/ImageDirectionDetection(DNN).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Image Direction Detection
- Problem : detect whether a given input image is correctly direction. 
 + Positive sample: not rotated/flipped, correctly oriented, `label=1`. 
 + Modified data to be a negative sample (flipped/rotated, `label=0`).

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import os
import datetime

## 1.Load Dataset

In [ ]:
# Loading raw
TRAIN_VAL_SPLIT = 0.15
SEED = 1337
IMG_SIZE = (256, 256)
BATCH = 16

###1.1 Pre-processing and Splitting

#### a, Copy dataset

In [ ]:
# Image data preprocessing
raw_dataset_train = tf.keras.utils.image_dataset_from_directory(
    directory="/content/drive/MyDrive/MachineMags/ImageDirection/Data/Images",
    labels=None, crop_to_aspect_ratio=True, seed=SEED, image_size=IMG_SIZE,
    validation_split=TRAIN_VAL_SPLIT, subset="training", batch_size=BATCH)
raw_dataset_val = tf.keras.utils.image_dataset_from_directory(
    directory="/content/drive/MyDrive/MachineMags/ImageDirection/Data/Images",
    labels=None, crop_to_aspect_ratio=True, seed=SEED, image_size=IMG_SIZE,
    validation_split=TRAIN_VAL_SPLIT, subset="validation", batch_size=BATCH)

#### b, Splitting Dataset with label to each class

In [ ]:
@tf.function
def add_modifications_and_labels(image_batch):
    batch_size = tf.shape(image_batch)[0]
    print('batch_size:' + str(batch_size))
    flipped_images = tf.image.flip_up_down(image_batch)

    rotated_images = tf.image.rot90(image_batch, k=1)
    print('rotated_images:' + str(rotated_images))
    original_label = tf.ones([batch_size], dtype=tf.bool)
    print('original_label:' + str(original_label))
    modified_label = tf.zeros([batch_size], dtype=tf.bool)
    print('modified_label:' + str(modified_label))

    # Concatenate flipped/rotated images and original/modified labels to the dataset
    image_batch = tf.concat([image_batch, flipped_images, rotated_images], 0)
    labels = tf.cast(tf.concat([original_label, modified_label, modified_label], 0), tf.int32)

    return image_batch, labels

dataset_train = raw_dataset_train.map(add_modifications_and_labels, num_parallel_calls=tf.data.AUTOTUNE)
dataset_val = raw_dataset_val.map(add_modifications_and_labels, num_parallel_calls=tf.data.AUTOTUNE)

#### b, Dataset visually

In [ ]:
def plot_images_and_labels(batch, predictions=None, stride=1, rows=4, cols=5):
    plt.figure(figsize=(4 * cols, 4 * rows))
    for images, labels in batch:
        print(labels)
        for i in range(rows * cols):
            ax = plt.subplot(rows, cols, i + 1)
            # display every nth image in the batch
            idx = stride * i
            image = images[idx]
            label = labels[idx].numpy()
            plt.imshow(image.numpy().astype("uint8"))
            title = "Label: " + str(label)
            title_color = 'black'
            if predictions is not None:
                prediction = predictions[idx]
                title += f", Pred: {prediction:.2f}"
                error = np.abs(label - prediction)
                if error > 0.5:
                    title_color = 'red'
            plt.title(title, color=title_color)
            plt.axis("off")
        break
    plt.show()

plot_images_and_labels(dataset_train.take(1), stride=2)

# 2. Train a Model
### Use a base model trained on ImageNet without head and apply transfer learning to fine-tune for this problem. We will use DNN Model

In [ ]:
# Load Pre-Trained Model EfficientNetB0
base_model = tf.keras.applications.efficientnet.EfficientNetB0(weights="imagenet", include_top=False,
                                                               input_tensor=tf.keras.layers.Input(shape=(256, 256, 3)))

In [ ]:
base_model.trainable = False
# Data augmentation
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip("horizontal"),
])

base_model.summary()

model = tf.keras.Sequential([
    base_model,
    data_augmentation,
    tf.keras.layers.GlobalMaxPooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=2e-4),
    loss=tf.losses.BinaryCrossentropy(),
    metrics=[tf.keras.metrics.BinaryAccuracy(), 'AUC', 'accuracy'])
model.summary()

## Model Training 
#### Train model with 30 epochs

In [ ]:
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

epochs = 30;
history = model.fit(
    dataset_train,
    validation_data=dataset_val,
    epochs=epochs,
    callbacks=[tensorboard_callback]
)

model.save('/content/drive/MyDrive/MachineMags/ImageDirection/img_orientation_detector_v1.h5')

In [ ]:
%load_ext tensorboard

In [ ]:
%tensorboard --logdir logs

## Prediction

In [ ]:
#sampled_images, sampled_labels = dataset_val.shuffle(1024).get_single_element()
sampled_images, sampled_labels = dataset_val.unbatch().shuffle(1024).batch(64).take(1).get_single_element()


print(len(sampled_images))
print(len(sampled_labels))

predictions = model.predict(sampled_images)
#
plot_images_and_labels([(sampled_images, sampled_labels)], predictions.squeeze(), stride=1, rows=5)

# 3. Evaluation
### Get Final Accuracy value

In [ ]:
res = model.evaluate(dataset_val)

###Visually the Training Result, Evaluation Metric

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()